### Create a file for the threhsold values for each sample/tower

In [1]:
# Set up the module for the porosity_analysis packages  

import os 
import sys 

Pyro_module  = '../20250312_PorosityAnalysis/porosity_analysis'
# Add the parent directory of Pyro_DataAnalysis to the system path
if os.path.exists(Pyro_module) == True:
    sys.path.insert(0, os.path.abspath(Pyro_module))
else :
    print('The file does not exist')

In [ ]:
#import statements 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
import os 
import numpy as np

from skimage.filters import difference_of_gaussians,gaussian
from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops,regionprops_table ,find_contours
from skimage.color import label2rgb
import matplotlib.patches as mpatches
import imageio.v3 as iio

import config 
import custom_funcs 
import image_analysis as ia

In [3]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import PIL.Image
PIL.Image.MAX_IMAGE_PIXELS = 3019898880

In [4]:
# Google drive location 
experimental_dir = config.technical_dir_path

In [48]:
# Save location
df_save_loc = os.path.join(experimental_dir,'data/processed/global_threshold.csv')

In [50]:
df_done = pd.read_csv(df_save_loc)
ref_im_done = list(df_done['ref_im'])

### Pre-processing in Fiji

#### Before anything can happen:
- The images should be cropped to just the relevant area
- Saved all images in .png in the processed folder 
- Put full images for thresholding in thresholding folder

### Reference image for thresholding 

In [51]:
ref_im_dir = os.path.join(experimental_dir,'data/processed/thresholding')
os.path.exists(ref_im_dir)

True

In [52]:
df = pd.DataFrame(columns= ['ref_im','date','Material','Designation','Otsu']) # Create Dataframe with the column headings

In [53]:

for ref_im in os.listdir(ref_im_dir):
    parts = ref_im.split("_")

    if ref_im in ref_im_done:
        continue
    
    if len(parts) >= 3: #check if the filename is in the correct format.
            date = parts[0]
            Material = parts[1]
            Designation = parts[2]

            # load image
            ref_img = os.path.join(ref_im_dir, ref_im)
            
            im = custom_funcs.load_image(ref_img)
            ## check if an image is greyscale
            gs_img = custom_funcs.is_greyscale(im)
            
            # Clean image 
            image_blur = gaussian(gs_img,2)
            # Get threshold value 
            Otsu = threshold_otsu(image_blur)
            after_Otsu = image_blur < Otsu


            # Append to DataFrame
            df = pd.concat([df, pd.DataFrame([{'ref_im': ref_im, 'date': date, 'Material': Material, 'Designation': Designation, 'Otsu': Otsu}])], ignore_index=True)
            



C:\Users\Neil\AppData\Local\Temp\ipykernel_14768\2015535755.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([{'ref_im': ref_im, 'date': date, 'Material': Material, 'Designation': Designation, 'Otsu': Otsu}])], ignore_index=True)


### Create df 

In [59]:
df.to_csv(df_save_loc, index=False, header=False, mode='a')

In [60]:
df

,ref_im,date,Material,Designation,Otsu
0,20250310_Nickel_02_00_thresh.png,20250310,Nickel,02,0.534619
1,20250310_Nickel_01_thresh.png,20250310,Nickel,01,0.537771
2,20250310_Nickel_05_thresh.png,20250310,Nickel,05,0.528401
3,20250310_Nickel_04_thresh.png,20250310,Nickel,04,0.534359
4,20250310_Nickel_03_thresh.png,20250310,Nickel,03,0.535766
5,20250219_Nickel_03_thresh.png,20250219,Nickel,03,0.420022
6,20250312_Nickel_02_thresh.png,20250312,Nickel,02,0.528459
7,20250312_Nickel_01_thresh.png,20250312,Nickel,01,0.401673
